In [0]:
%matplotlib inline

import math
from math import sin, cos, tan, asin, acos, atan2, pi
import random

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def normalize(v):
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

LENGTH = 1
SEGMENT_COUNT = 200
ANGLE = math.radians(140)
PPO_ANGLE = math.radians(109.5)
i_vector = np.array([1, 0, 0])

assert ANGLE > pi / 2
assert ANGLE < pi

r = LENGTH
theta = pi - ANGLE

random.seed()

def calc():
    # points = [[0.1, 0.1, math.sqrt(0.98)], [0, 0, 0]]
    points = [np.array([0, 0, 0]), np.array([0.1, 0.1, math.sqrt(0.98)])]
    ppo_chains = []
    for index in range(SEGMENT_COUNT):
        prev_point = points[index]
        curr_point = points[index + 1]
        prev_vector = curr_point - prev_point
        circle_center = curr_point + prev_vector * cos(pi - ANGLE)
        circle_radius = LENGTH * sin(ANGLE)

        u = np.cross(prev_vector, i_vector)
        u = u / np.linalg.norm(u, ord=1)
        v = normalize(np.cross(prev_vector, u))
        t = random.uniform(0, 2 * pi)
        next_point = circle_center + circle_radius * cos(t) * u + circle_radius * sin(t) * v
        points.append(next_point)

        if random.random() < 1 / 30:
            ppo_points = []
            circle_center = curr_point + prev_vector * cos(pi - 90)
            circle_radius = LENGTH * sin(90)
            u = np.cross(prev_vector, i_vector)
            u = u / np.linalg.norm(u, ord=1)
            v = normalize(np.cross(prev_vector, u))
            t = random.uniform(0, 2 * pi)
            next_point = circle_center + circle_radius * cos(t) * u + circle_radius * sin(t) * v
            ppo_points.append(curr_point)
            ppo_points.append(next_point)

            for ppo_index in range(10):
                prev_point = ppo_points[ppo_index]
                curr_point = ppo_points[ppo_index + 1]
                prev_vector = curr_point - prev_point
                circle_center = curr_point + prev_vector * cos(pi - PPO_ANGLE)
                circle_radius = LENGTH * sin(PPO_ANGLE)

                u = np.cross(prev_vector, i_vector)
                u = u / np.linalg.norm(u, ord=1)
                v = normalize(np.cross(prev_vector, u))
                t = random.uniform(0, 2 * pi)
                next_point = circle_center + circle_radius * cos(t) * u + circle_radius * sin(t) * v
                ppo_points.append(next_point)

            ppo_chains.append(ppo_points)
            
    
    squared_dist = np.sum(points[0]**2 + points[-1]**2, axis=0)
    distance = np.sqrt(squared_dist)
    return points, ppo_chains, distance

def draw():
    points, ppo_chains, distance = calc()
    fig = plt.figure(figsize=[12, 7], dpi=80)
    ax = fig.add_subplot(111, projection='3d')

    for idx, point in enumerate(points):
        ax.scatter(*point, c='r', marker='o')
        if idx != 0:
            ax.plot([points[idx - 1][0], points[idx][0]],
                    [points[idx - 1][1], points[idx][1]],
                    zs=[points[idx - 1][2], points[idx][2]],
                    color='red')

    for chain in ppo_chains:
        for idx, point in enumerate(chain):
            ax.scatter(*point, c='g', marker='o')
            if idx != 0:
                ax.plot([chain[idx - 1][0], chain[idx][0]],
                        [chain[idx - 1][1], chain[idx][1]],
                        zs=[chain[idx - 1][2], chain[idx][2]],
                        color='green')

    # SCALE = 30
    # ax.set_xlim3d([-SCALE, SCALE])
    # ax.set_ylim3d([-SCALE, SCALE])
    # ax.set_zlim3d([-SCALE, SCALE])

    min_x = min(a[0] for a in points)
    min_y = min(a[1] for a in points)
    min_z = min(a[2] for a in points)
    max_x = max(a[0] for a in points)
    max_y = max(a[1] for a in points)
    max_z = max(a[2] for a in points)
    avg_x = np.mean([a[0] for a in points])
    avg_y = np.mean([a[1] for a in points])
    avg_z = np.mean([a[2] for a in points])
    max_scale_length = max([max_x - min_x, max_y - min_y, max_z - min_z])
    ax.set_xlim3d([avg_x - 0.25 * max_scale_length, avg_x + 0.25 * max_scale_length])
    ax.set_ylim3d([avg_y - 0.25 * max_scale_length, avg_y + 0.25 * max_scale_length])
    ax.set_zlim3d([avg_z - 0.25 * max_scale_length, avg_z + 0.25 * max_scale_length])

    # plt.tight_layout()
    # plt.show()
    # import time
    # for ii in range(5):
    #     ax.view_init(elev=10., azim=ii)
    #     plt.draw()
    #     time.sleep(1)
    
    return distance
  
distances = []

In [0]:
distance = draw()
distances.append(distance)
print('R = {}'.format(distance))

In [0]:
plt.hist(distances, bins=10)

In [0]:
distances = []
for _ in range(1000):
    _, _, distance = calc()
    distances.append(distance)
plt.hist(distances, bins=20)